## VARS Darwin Core conversion - initial data set (6 species, all dates)

Resources:
- https://dwc.tdwg.org/terms/
- https://tools.gbif.org/dwca-validator/extension.do?id=dwc:Occurrence
- https://www.mbari.org/products/research-software/video-annotation-and-reference-system-vars/query-interface/advanced-user-guide/
- https://www.gbif.org/data-quality-requirements-occurrences

In [48]:
## Imports

import pandas as pd
import numpy as np

from datetime import datetime # for handline dates
import pytz # for handling time zones

import urllib.request, urllib.parse, json # for dealing with WoRMS API and output
import WoRMS # functions for querying WoRMS REST API

In [49]:
## Load csv

path = ''
filename = 'VARS_DwC_conversion_practice_200403.csv'
data = pd.read_csv(path+filename)

data.head()

,imaged_moment_uuid,index_elapsed_time_millis,index_recorded_timestamp,index_timecode,observation_uuid,activity,concept,duration_millis,observation_group,observation_timestamp,...,video_description,video_duration_millis,video_name,video_start_timestamp,camera_id,video_sequence_description,video_sequence_name,chief_scientist,dive_number,camera_platform
0,97BD5895-9489-478B-8797-6961D8A770D2,NaN,2001-03-19 22:24:12,04:00:43:27,1040A0BF-2D50-45D2-9C96-35CC80708939,cruise,Dosidicus gigas,NaN,ROV,2010-10-25 22:10:05.150000,...,NaN,5529000.0,T0265-04,2001-03-19 20:57:56,Tiburon,NaN,Tiburon 0265,Bruce Robison,Tiburon 0265,Tiburon
1,97BD5895-9489-478B-8797-6961D8A770D2,NaN,2001-03-19 22:24:12,04:00:43:27,1040A0BF-2D50-45D2-9C96-35CC80708939,cruise,Dosidicus gigas,NaN,ROV,2010-10-25 22:10:05.150000,...,NaN,5529000.0,T0265-04,2001-03-19 20:57:56,Tiburon,NaN,Tiburon 0265,Bruce Robison,Tiburon 0265,Tiburon
2,97BD5895-9489-478B-8797-6961D8A770D2,NaN,2001-03-19 22:24:12,04:00:43:27,1040A0BF-2D50-45D2-9C96-35CC80708939,cruise,Dosidicus gigas,NaN,ROV,2010-10-25 22:10:05.150000,...,NaN,5529000.0,T0265-04,2001-03-19 20:57:56,Tiburon,NaN,Tiburon 0265,Bruce Robison,Tiburon 0265,Tiburon
3,BE6C5C5B-8B04-45F0-BD48-C3EBE187B2EB,NaN,2004-05-03 17:27:02,03:24:58:15,A3DE4416-5B67-4A10-A6BC-AE9A99552A13,cruise,Dosidicus gigas,NaN,ROV,2004-05-03 18:29:03,...,NaN,3600000.0,T0666-04,2004-05-03 17:12:42,Tiburon,NaN,Tiburon 0666,David Clague,Tiburon 0666,Tiburon
4,DBCD3F40-E533-4AD0-9541-A87932C9EC78,NaN,2010-02-25 18:58:34,00:38:45:13,BF43A883-1DE2-439A-8F55-F7F215FE2D4F,cruise,Dosidicus gigas,NaN,ROV,2010-04-14 23:40:00.360000,...,NaN,3596000.0,V3527-01HD,2010-02-25 18:40:18,Ventana,NaN,Ventana 3527,Linda Kuhnz,Ventana 3527,Ventana


In [50]:
## List all columns

data.columns

Index(['imaged_moment_uuid', 'index_elapsed_time_millis',
       'index_recorded_timestamp', 'index_timecode', 'observation_uuid',
       'activity', 'concept', 'duration_millis', 'observation_group',
       'observation_timestamp', 'observer', 'image_reference_uuid',
       'image_description', 'image_format', 'image_height', 'image_width',
       'image_url', 'link_name', 'link_value', 'to_concept',
       'association_mime_type', 'associations', 'altitude',
       'coordinate_reference_system', 'depth_meters', 'latitude', 'longitude',
       'oxygen_ml_per_l', 'phi', 'xyz_position_units', 'pressure_dbar', 'psi',
       'salinity', 'temperature_celsius', 'theta', 'x', 'y', 'z',
       'light_transmission', 'video_reference_uuid', 'audio_codec',
       'video_container', 'video_reference_description', 'frame_rate',
       'video_height', 'video_sha512', 'video_size_bytes', 'video_uri',
       'video_codec', 'video_width', 'video_description',
       'video_duration_millis', 'video_nam

### What do all these columns mean?

I don't think all of them are in the user guide, even the advanced user guide. They also don't align with the columns listed in Brian's SQL code, for some reason. **They don't align with Brian's code because that code was written for a legacy version of the database.**

Below:
- **bold** = term or important observation
- <span style="color:red">**red**</span> = questions still to ask
- <span style="color:orange">**orange**</span> = things I still don't understand but it probably doesn't matter

imaged_moment_uuid = unique identifier for the imaged_moments table. All information about images and annotations is stored in the M3_ANNOTATIONS database.<br>
imaged_elapsed_time_millis = how long the video had been running when the image was captured <br>
index_recorded_timestamp = **Recorded Date**, the time in UTC when the image was captured on camera <br>
index_timecode = **Tape Time Code**, the hours, minutes, seconds and frames since the beginning of the dive (00:00:00:00) in the format (HH:MM:SS:FF); used for tape <br>
observation_uuid = the unique id of the annotation, unique identifier for the observations table. **For one imaged moment, there can be multiple observations.** <br>
activity = **Camera Direction**, describes what the ROV was doing when the image was taken, possible values: 'cruise', 'descend', 'ascend', 'transect', 'stationary', 'unspecified', 'diel transect', nan <br>
concept = **Concept Name**, key terms referring to organisms, geologic features, sampling devices/scientific equipment, and marine debris - I assume we're interested in organisms only <br>
duration_millis = how long the concept was observed for, starting from index_recorded_timestamp. <br>
observation_group = only has a value of ROV in this sample data set <br>
observation_timestamp = **Observation Date**, the date the annotation was created in UTC, not necessarily the same as Recorded Date <br>
observer = **Observer**, the mbari username of the person who created the annotation in theory, but in practice a mix of usernames, full names, partial names, different capitalizations, etc. <br>
image_reference_uuid = unique id for the image_reference table, which contains info about the size, type of image <br>
image_description = each imaged_moment is saved at least twice in compressed and uncompressed format; takes on values of 'compressed image with overlay', 'source image', 'uncompressed image', 'compressed image', nan <br>
image_format = format of associated image file: 'image/jpg', 'image/png', nan. One of these should correspond to compressed and one to uncompressed description. <br>
image_height = height of associated image file in pixels: 0, 1080, nan. <span style="color:red">**What does 0 mean versus nan in this context?**</span> <br>
image_width = width of associated image file in pixels: 0, 1920, nan. <span style="color:red">**What does 0 mean versus nan in this context?**</span> <br>
image_url = url where the associated image is stored online. <span style="color:red">**These links are available to everyone, including outside MBARI**</span> <br>
link_name = to the best of my understanding, link_name and link_value are like a dictionary key, value pair. The link_name indicates what kind of information is held in link_value. Some of these are clearly descriptors of the animal or its behavior, others are confusing (e.g. "on"). <span style="color:red">**Difference between nil and nan?**</span> **Each observation can have multiple associations (i.e. link_names, values, etc.)** <br>
link_value = value of the information described by link_name. <br>
to_concept = points to 'self' or to another concept. So, for example, if the annotation is for a red octopus, the concept might be the octopus species name, the link_name might be 'color', the to_concept might be 'self', and the link_value might be 'red'. <span style="color:red">**Difference between nil and nan?**</span> <br>
<span style="color:orange">association_mime_type</span> - either 'text/plain' or nan, possibly the data type of the link_value? <br>
<span style="color:orange">associations</span> <br>
altitude = how far the ROV was off the bottom? In meters? <span style="color:orange">**What do the negative values mean?**</span> <br>
coordinate_reference_system = all nan here, **Assume WGS84, but Brian wasn't sure and wanted to double check.** <br>
depth = depth below the surface in meters, positive number, ranges from ~6 to ~3500 here. <br>
latitude = latitude where the image was taken in decimal degrees <br>
longitude = longitude where the image was taken in decimal degrees <br>
oxygen_ml_per_l = **Oxygen**, mL of dissolved oxygen per L seawater, includes nan values, ranges from -1.5 to 15 <br>
<span style="color:orange">phi</span> <br>
<span style="color:orange">xyz_position_units</span> - all nan for these data <br>
pressure_dbar = the pressure measured in decibars at the time the image was taken <br>
psi = <span style="color:red">**The same pressure converted to psi?**</span> <br>
salinity = **Salinity**, the salinity at the time the image was taken, calculated from conductivity and pressure measurements, or nan <br>
temperature_celcius = **Temperature**, the water temperature in degrees C when the image was taken, or nan <br>
<span style="color:orange">theta</span> <br>
<span style="color:orange">x</span> <br>
<span style="color:orange">y</span> <br>
<span style="color:orange">z</span> <br>
light_transmission = **Light**, percent light transmitted through the water column when the image was taken, or nan. **Patrick suggests not including this as a MeasurementOrFact, because the ROVs generate light and so it's unclear what's actually being measured.** <br>
video_reference_uuid = unique id in video_references table, which holds information about the size, format, etc. of video files. All video data are stored in the M3_VIDEO_ASSETS database. <br>
<span style="color:orange">audio_codec</span> - all nan for these data <br>
video_container = whether the original video exists on tape or digitally, options are 'tape', 'video/quicktime' <br>
<span style="color:orange">video_reference_description</span> - appears to be the words "Tape loaded from VARS on" plus a datetime <br>
frame_rate = frame rate of the camera, either 29.97 or 0 <span style="color:orange">**What does 0 mean in this context? It's a still image?**</span> <br>
video_height = height of the video frame in pixels, 1080 <br>
<span style="color:orange">video_sha512</span> <br>
video_size_bytes = size of the video file in bytes, 0.00000000e+00, 2.62291551e+10, 2.67221996e+10; larger videos are probably quicktime files <br>
video_uri = location of video file on MBARI's servers, or a pointer to where the tape is stored. If you want to open a video on the web, make sure you're VPN'ed into MBARI and choose an mp4 rather than a quicktime file. **These files are not currently publically accessible.**
<span style="color:orange">video_codec</span> - all nan for these data <br>
video_width = width of the video frame in pixels, 1920 <br>
<span style="color:orange">video_description</span> - all nan for these data <br>
video_duration_millis = total duration of the video in milliseconds. **Multiple videos are taken per dive.** <br>
<span style="color:orange">video_name</span> <br>
video_start_timestamp = time when the video started <span style="color:orange">**in local time?**</span> <br>
camera_id = **ROV name**, 'Tiburon', 'Ventana', or 'Doc Ricketts' <br>
<span style="color:orange">video_sequence_description</span> - all nan for these data <br>
video_sequence_name = ROV name plus a 4-digit integer (dive number); indicates which video sequence/dive a particular video came from. **Use this instead of dive_number, it is more up to date.** <br>
chief_scientist = **Chief Scientist**, the full name of PI for whom the dive was primarily conducted, maiden name sometimes included in parentheses <br>
dive_number = **Dive Number**, ROV name plus a 4-digit integer uniquely identifying which dive a video/image was taken on, **Out of date, use video_sequence_name** <br>
camera_platform = **ROV name**, 'Tiburon', 'Ventana', 'Doc Ricketts' or nan, <span style="color:red">**How is this different than camera_id? What does nan mean in this context?**</span>

### How might these map to DwC terms?

**Event** = dive <br>
**Occurrence** = annotated observation

We will create two files for submission to OBIS, one containing event information and one containing occurrence information. Then, we can combine these into a single file as needed for compatibility with ERDDAP.

#### Event file contents
index_recorded_timestamp = **eventDate** in UTC, ISO 8601:2004. Extract only date. <br>
observation_group = 'ROV', some or all of **samplingProtocol**? <br>
video_sequence_name = **eventID** <br>
chief_scientist = **recordedBy** <br>

#### Occurrence file contents
index_recorded_timestamp = **eventDate** in UTC, ISO 8601:2004 <br>
observation_uuid = **occurrenceID**. Must be mindful that there may be multiple rows with the same observation_uuid due to multiple available images, multiple available videos, and/or multiple associations if using data from the associations table. <span style="color:red">Note that, at least for the first go, I'll probably leave out information from the associations table.</span> <br>
concept = **scientificName, scientificNameID, taxonID, nameAccordingToID, identificationReferences, occurrenceStatus, basisOfRecord** <br>
observer = **identifiedBy**. Also assign **institutionCode** as MBARI. <br>
image_url = **associatedMedia**. Note that we may need to choose which image to link to, or there may be a way to link to multiple images and/or videos. <br>
depth = **minimumDepthInMeters, maximumDepthInMeters**. <br>
latitude = **decimalLatitude** <br>
longitude = **decimalLongitude** <br>
oxygen_ml_per_l = a MeasurementOrFact, **measurementType, measurementValue, measurementAccuracy, measurementUnits**. Note that if there is more than one MeasurementOrFact to be included, create sensible column names for each and then designate those columns as MeasurementOrFact columns in the metadata file. <br>
psi, salinity, temperature_celcius, light_transmission = other candidates for MeasurementOrFact <br>
video_uri = **associatedMedia**. Again, we may need to figure out how to deal with multiple media links, and which to provide if not all of them. <br>
video_sequence_name = **eventID** <br>

#### Maybe have some potential to be included in occurrence file:
link_value <br>
link_name <br>
to_concept <br>

In [51]:
## Get data with only relevant columns

df = data[['index_recorded_timestamp', 'observation_group', 'video_sequence_name', 'chief_scientist', 'observation_uuid', 'concept', 'observer', 'image_url', 'depth_meters', 
          'latitude', 'longitude', 'oxygen_ml_per_l', 'psi', 'salinity', 'temperature_celsius', 'video_uri']]
df.head()

,index_recorded_timestamp,observation_group,video_sequence_name,chief_scientist,observation_uuid,concept,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri
0,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04
1,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04
2,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04
3,2004-05-03 17:27:02,ROV,Tiburon 0666,David Clague,A3DE4416-5B67-4A10-A6BC-AE9A99552A13,Dosidicus gigas,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2004/124/03_25_01_14.png,546.400024,32.273680,-119.672170,0.380,70.800003,34.199001,5.580,urn:tid:mbari.org:T0666-04
4,2010-02-25 18:58:34,ROV,Ventana 3527,Linda Kuhnz,BF43A883-1DE2-439A-8F55-F7F215FE2D4F,Dosidicus gigas,linda,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3527/00_38_45_13.jpg,441.980011,36.797201,-122.188644,0.854,195.199997,34.175999,6.959,urn:tid:mbari.org:V3527-01HD


### Where do multiple rows from the with the same observation ID come from?

In [52]:
df[df['observation_uuid'] == df['observation_uuid'].iloc[0]]

,index_recorded_timestamp,observation_group,video_sequence_name,chief_scientist,observation_uuid,concept,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri
0,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04
1,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04
2,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04
3772,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04
3784,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04
3791,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04


In [53]:
pd.options.display.max_colwidth = 100 # default=50
df.loc[df['observation_uuid'] == df['observation_uuid'].iloc[0], 'image_url']

0       http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg
1       http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg
2       http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg
3772    http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png
3784    http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png
3791    http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png
Name: image_url, dtype: object

Ok, so two of these duplicates are arising from the compressed (.jpg) and uncompressed (.png) versions of the image files. I bet the rest of them are coming from associations information that I haven't included in df. Let's check that.

In [54]:
data.loc[data['observation_uuid'] == data['observation_uuid'].iloc[0], ['index_recorded_timestamp', 'observation_uuid', 'concept', 'image_description', 'image_url', 
                                                                       'link_name', 'link_value', 'to_concept', 'video_uri', 'video_size_bytes']]

,index_recorded_timestamp,observation_uuid,concept,image_description,image_url,link_name,link_value,to_concept,video_uri,video_size_bytes
0,2001-03-19 22:24:12,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,compressed image with overlay,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,perspective,close-up,self,urn:tid:mbari.org:T0265-04,0.0
1,2001-03-19 22:24:12,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,compressed image with overlay,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,identity-reference,4,self,urn:tid:mbari.org:T0265-04,0.0
2,2001-03-19 22:24:12,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,compressed image with overlay,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,identity-certainty,maybe,self,urn:tid:mbari.org:T0265-04,0.0
3772,2001-03-19 22:24:12,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,source image,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png,perspective,close-up,self,urn:tid:mbari.org:T0265-04,0.0
3784,2001-03-19 22:24:12,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,source image,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png,identity-reference,4,self,urn:tid:mbari.org:T0265-04,0.0
3791,2001-03-19 22:24:12,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,source image,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png,identity-certainty,maybe,self,urn:tid:mbari.org:T0265-04,0.0


That seems right. So, it seems like if we filter for unique observation ID's within compressed images only, that should work. Although, we may want to provide the links to both images... In that case, we could:
1. Eliminate link_name, link_value, etc. from the data frame
2. Drop duplicates
3. Get multiple rows of associated media information (e.g. .jpg and .png format image links) into a single column of the form image_url_1 | image_url_2 | video_uri

**Note** that I just tried to open an image link while not VPN'ed in to MBARI, and got:

Forbidden: You don't have permission to access /ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg on this server.

Are we sure these files are publically accessible?

In [55]:
## Drop duplicate rows from df

df = df.drop_duplicates()
df.head()

,index_recorded_timestamp,observation_group,video_sequence_name,chief_scientist,observation_uuid,concept,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri
0,2001-03-19 22:24:12,ROV,Tiburon 0265,Bruce Robison,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,urn:tid:mbari.org:T0265-04
3,2004-05-03 17:27:02,ROV,Tiburon 0666,David Clague,A3DE4416-5B67-4A10-A6BC-AE9A99552A13,Dosidicus gigas,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2004/124/03_25_01_14.png,546.400024,32.273680,-119.672170,0.380,70.800003,34.199001,5.580,urn:tid:mbari.org:T0666-04
4,2010-02-25 18:58:34,ROV,Ventana 3527,Linda Kuhnz,BF43A883-1DE2-439A-8F55-F7F215FE2D4F,Dosidicus gigas,linda,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3527/00_38_45_13.jpg,441.980011,36.797201,-122.188644,0.854,195.199997,34.175999,6.959,urn:tid:mbari.org:V3527-01HD
6,2007-11-02 20:22:18,ROV,Tiburon 1147,Bruce Robison,0EFEFAFD-B9ED-4640-9873-75A5EDA7C253,Muusoctopus robustus,svonthun,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/1147/06_40_40_00.jpg,3411.820068,36.323256,-122.903316,2.650,139.000000,34.484001,1.589,urn:tid:mbari.org:T1147-07
7,2010-02-24 17:59:12,ROV,Ventana 3525,Chad Widmer,2E2280EF-397A-4384-B94B-23B12A287175,Dosidicus gigas,lonny,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3525/01_06_04_24.jpg,454.769989,36.714396,-121.999731,0.828,99.300003,34.181000,6.962,urn:tid:mbari.org:V3525-02HD


In [56]:
df.loc[df['observation_uuid'] == df['observation_uuid'].iloc[0], 'image_url']

0       http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg
3772    http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.png
Name: image_url, dtype: object

**Initially, we converted df into separate event and occurrence dataframes. After more thought, we decided to only submit a single dataframe, and see what feedback we get. The changes below reflect this.**

### Assemble event data

In [57]:
## Start with basic event data

converted = df[['index_recorded_timestamp', 'video_sequence_name', 'observation_group', 'chief_scientist']]
converted.head()

,index_recorded_timestamp,video_sequence_name,observation_group,chief_scientist
0,2001-03-19 22:24:12,Tiburon 0265,ROV,Bruce Robison
3,2004-05-03 17:27:02,Tiburon 0666,ROV,David Clague
4,2010-02-25 18:58:34,Ventana 3527,ROV,Linda Kuhnz
6,2007-11-02 20:22:18,Tiburon 1147,ROV,Bruce Robison
7,2010-02-24 17:59:12,Ventana 3525,ROV,Chad Widmer


In [58]:
## Change headings

converted = converted.rename(columns={
    'index_recorded_timestamp':'eventDate',
    'video_sequence_name':'eventID',
    'observation_group':'samplingProtocol',
    'chief_scientist':'recordedBy'
})
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-03-19 22:24:12,Tiburon 0265,ROV,Bruce Robison
3,2004-05-03 17:27:02,Tiburon 0666,ROV,David Clague
4,2010-02-25 18:58:34,Ventana 3527,ROV,Linda Kuhnz
6,2007-11-02 20:22:18,Tiburon 1147,ROV,Bruce Robison
7,2010-02-24 17:59:12,Ventana 3525,ROV,Chad Widmer


In [59]:
## Remove whitespace from eventID

converted['eventID'] = [event.replace(' ', '_') for event in converted['eventID']]
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-03-19 22:24:12,Tiburon_0265,ROV,Bruce Robison
3,2004-05-03 17:27:02,Tiburon_0666,ROV,David Clague
4,2010-02-25 18:58:34,Ventana_3527,ROV,Linda Kuhnz
6,2007-11-02 20:22:18,Tiburon_1147,ROV,Bruce Robison
7,2010-02-24 17:59:12,Ventana_3525,ROV,Chad Widmer


**Note** that this code also places an underscore between 'Doc' and 'Ricketts'. It's possible that using 'DocRicketts' could be preferable.

In [60]:
## Add institutionCode

converted['institutionCode'] = 'MBARI'
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode
0,2001-03-19 22:24:12,Tiburon_0265,ROV,Bruce Robison,MBARI
3,2004-05-03 17:27:02,Tiburon_0666,ROV,David Clague,MBARI
4,2010-02-25 18:58:34,Ventana_3527,ROV,Linda Kuhnz,MBARI
6,2007-11-02 20:22:18,Tiburon_1147,ROV,Bruce Robison,MBARI
7,2010-02-24 17:59:12,Ventana_3525,ROV,Chad Widmer,MBARI


**Note** that the dataframe currently contains duplicate rows due to multiple image_urls not yet included. We will handle these later.

In [61]:
## Format eventDate

formatted = []

for dt in converted['eventDate']:
    
    # Convert string to datetime
    try:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S.%f') # some datetimes have milliseconds
    except ValueError:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
        
    # Assign UTC timezone
    utc = pytz.UTC
    dt = dt.astimezone(utc)
    
    # Put in ISO format string
    dt = dt.isoformat()
    
    # Save in list
    formatted.append(dt)

converted['eventDate'] = formatted
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode
0,2001-03-20T05:24:12+00:00,Tiburon_0265,ROV,Bruce Robison,MBARI
3,2004-05-04T00:27:02+00:00,Tiburon_0666,ROV,David Clague,MBARI
4,2010-02-26T02:58:34+00:00,Ventana_3527,ROV,Linda Kuhnz,MBARI
6,2007-11-03T03:22:18+00:00,Tiburon_1147,ROV,Bruce Robison,MBARI
7,2010-02-25T01:59:12+00:00,Ventana_3525,ROV,Chad Widmer,MBARI


In [62]:
## Check for any weird values in recordedBy

converted['recordedBy'].unique()

array(['Bruce Robison', 'David Clague', 'Linda Kuhnz', 'Chad Widmer',
       'Patrick Whaling', 'Ken Smith', 'Bob Vrijenhoek',
       'Shannon Johnson', 'Joe Jones', 'Chuck Saltsman', 'Edie Widder',
       'Steve Haddock', 'Larry Madin', 'Steve Rock', 'Peter Brewer',
       'Andrew DeVogelaere', 'Jim Barry', 'Lisa Levin', nan,
       'Rob Sherlock', 'John Orcutt', 'Kim Reisenbichler', 'Craig Smith',
       'Gary Greene', 'Alastair Fothergill', 'Peter Walz',
       'Paul Schrader', 'William Chadwick', 'Erika Raymond',
       'Alana Sherman', 'Larry Bird', 'Charlie Paull', 'Dave Caress',
       'Craig McClain', 'Paul McGill', 'Meghan Powers', 'Debra Stakes',
       'Keenan Ball', 'Sebastian Sudek', 'Mark Chaffey', 'Scott Wankel',
       'Stephanie Bush', 'Sheri White', 'Craig Dawe', 'Peter Girguis',
       'Tony Ramirez', 'Russ Hopcroft', 'Michael Begnaud', 'Chuck Baxter',
       'Douglas Pargett', 'Bill Ussler', 'Knute Brekke', 'Luke Coletti',
       'Nancy Jacobsen', 'Lou Zeidberg', 'C

### Datetimes and JDBC

Brian always saves timestamps in UTC, but apparently JayDeBeApi changes the timezones of datetime data when you use it to retrieve data from a database: <br>
https://github.com/baztian/jaydebeapi/issues/73

I just edited `_jdbc_connect_jpype` in JayDeBeApi's `__init__` function as instructed by one of the posts from this link:
```python
def _jdbc_connect_jpype(jclassname, url, driver_args, jars, libs):
    import jpype
    if not jpype.isJVMStarted():
        args = []
        class_path = []
        if jars:
            class_path.extend(jars)
        class_path.extend(_get_classpath())
        if class_path:
            args.append('-Djava.class.path=%s' %
                        os.path.pathsep.join(class_path))
        if libs:
            # path to shared libraries
            libs_path = os.path.pathsep.join(libs)
            args.append('-Djava.library.path=%s' % libs_path)
        args.append('-Duser.timezone=GMT') # ADD THIS LINE TO CODE
```

And re-pulled the data. They are saved as **VARS_DwC_conversion_practice_200416.csv**

#### Did this change actually change the datetimes?

In [63]:
## Load csv

path = ''
filename = 'VARS_DwC_conversion_practice_200416.csv'
new_times = pd.read_csv(path+filename)

# Get relevant columns
new_times = new_times[['index_recorded_timestamp', 'observation_group', 'video_sequence_name', 'chief_scientist', 'observation_uuid', 'concept', 'observer', 'image_url', 'depth_meters', 
          'latitude', 'longitude', 'oxygen_ml_per_l', 'psi', 'salinity', 'temperature_celsius', 'video_uri']]

# Drop duplicates
new_times = new_times.drop_duplicates()

# Examine
new_times.loc[new_times['observation_uuid'] == new_times['observation_uuid'].iloc[0]]

,index_recorded_timestamp,observation_group,video_sequence_name,chief_scientist,observation_uuid,concept,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri
0,2005-08-23 22:52:40,ROV,Tiburon 0884,Bob Vrijenhoek,A90437A1-F8F8-45C2-9862-3B54B0D036BF,Muusoctopus robustus,linda,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/0884/07_38_49_28.jpg,2755.620117,42.755189,-126.70974,1.24,136.699997,34.595001,1.756,urn:tid:mbari.org:T0884-08
2136,2005-08-23 22:52:40,ROV,Tiburon 0884,Bob Vrijenhoek,A90437A1-F8F8-45C2-9862-3B54B0D036BF,Muusoctopus robustus,linda,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/0884/07_38_49_28.png,2755.620117,42.755189,-126.70974,1.24,136.699997,34.595001,1.756,urn:tid:mbari.org:T0884-08


In [64]:
df.loc[df['observation_uuid'] == new_times['observation_uuid'].iloc[0]]

,index_recorded_timestamp,observation_group,video_sequence_name,chief_scientist,observation_uuid,concept,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri
883,2005-08-23 22:52:40,ROV,Tiburon 0884,Bob Vrijenhoek,A90437A1-F8F8-45C2-9862-3B54B0D036BF,Muusoctopus robustus,linda,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/0884/07_38_49_28.png,2755.620117,42.755189,-126.70974,1.24,136.699997,34.595001,1.756,urn:tid:mbari.org:T0884-08
3102,2005-08-23 22:52:40,ROV,Tiburon 0884,Bob Vrijenhoek,A90437A1-F8F8-45C2-9862-3B54B0D036BF,Muusoctopus robustus,linda,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/0884/07_38_49_28.jpg,2755.620117,42.755189,-126.70974,1.24,136.699997,34.595001,1.756,urn:tid:mbari.org:T0884-08


Changing the code as suggested did **NOT** actually alter the dates or times at all. I've tried entering a few timezones, including 'UTC', 'GMT', and 'PST', but the output is always the same. Could it be that this bug has been fixed?

Is there any other way to double check this?

Maybe I can compare to results I get using the VARS query app.

In [65]:
## Get records from original csv file (200403) matching a record found using the VARS query app

df.loc[df['observation_uuid'] == '67E049CE-45C1-4D1C-92C0-AB4890F77311']

,index_recorded_timestamp,observation_group,video_sequence_name,chief_scientist,observation_uuid,concept,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri
1493,2009-09-14 19:00:36,ROV,Ventana 3415,Steve Haddock,67E049CE-45C1-4D1C-92C0-AB4890F77311,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3415/02_49_08_24.jpg,539.659973,36.702458,-122.052307,0.447,336.600006,34.206001,5.85,urn:tid:mbari.org:V3415-03HD
3761,2009-09-14 19:00:36,ROV,Ventana 3415,Steve Haddock,67E049CE-45C1-4D1C-92C0-AB4890F77311,Dosidicus gigas,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3415/02_49_08_24.png,539.659973,36.702458,-122.052307,0.447,336.600006,34.206001,5.85,urn:tid:mbari.org:V3415-03HD


This timestamp matches what I found on the VARS query app... so maybe it's all OK? **Yes, I think our datetimes are fine. As per Henry's suggestion, I checked whether the timestamps matched those on the actual image file, and they do.**

### Add occurrence data

In [66]:
## Add in columns from df, renaming as needed

converted['occurrenceID'] = df['observation_uuid']
converted['scientificName'] = df['concept']
converted['identifiedBy'] = df['observer']
converted['minimumDepthInMeters'] = df['depth_meters']
converted['maximumDepthInMeters'] = df['depth_meters']
converted['decimalLatitude'] = df['latitude']
converted['decimalLongitude'] = df['longitude']
converted['dissolvedOxygenInMLPerL'] = df['oxygen_ml_per_l']
converted['pressureInPsi'] = df['psi']
converted['salinity'] = df['salinity']
converted['temperatureInCelsius'] = df['temperature_celsius']
converted['image_url'] = df['image_url']
converted['video_uri'] = df['video_uri']
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,minimumDepthInMeters,maximumDepthInMeters,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri
0,2001-03-20T05:24:12+00:00,Tiburon_0265,ROV,Bruce Robison,MBARI,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,718.200012,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,urn:tid:mbari.org:T0265-04
3,2004-05-04T00:27:02+00:00,Tiburon_0666,ROV,David Clague,MBARI,A3DE4416-5B67-4A10-A6BC-AE9A99552A13,Dosidicus gigas,vars,546.400024,546.400024,32.273680,-119.672170,0.380,70.800003,34.199001,5.580,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2004/124/03_25_01_14.png,urn:tid:mbari.org:T0666-04
4,2010-02-26T02:58:34+00:00,Ventana_3527,ROV,Linda Kuhnz,MBARI,BF43A883-1DE2-439A-8F55-F7F215FE2D4F,Dosidicus gigas,linda,441.980011,441.980011,36.797201,-122.188644,0.854,195.199997,34.175999,6.959,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3527/00_38_45_13.jpg,urn:tid:mbari.org:V3527-01HD
6,2007-11-03T03:22:18+00:00,Tiburon_1147,ROV,Bruce Robison,MBARI,0EFEFAFD-B9ED-4640-9873-75A5EDA7C253,Muusoctopus robustus,svonthun,3411.820068,3411.820068,36.323256,-122.903316,2.650,139.000000,34.484001,1.589,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/1147/06_40_40_00.jpg,urn:tid:mbari.org:T1147-07
7,2010-02-25T01:59:12+00:00,Ventana_3525,ROV,Chad Widmer,MBARI,2E2280EF-397A-4384-B94B-23B12A287175,Dosidicus gigas,lonny,454.769989,454.769989,36.714396,-121.999731,0.828,99.300003,34.181000,6.962,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3525/01_06_04_24.jpg,urn:tid:mbari.org:V3525-02HD


In [67]:
## Add coordinateUncertaintyInMeters 

converted['coordinateUncertaintyInMeters'] = round(converted['minimumDepthInMeters']*0.03, 2)
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,minimumDepthInMeters,maximumDepthInMeters,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters
0,2001-03-20T05:24:12+00:00,Tiburon_0265,ROV,Bruce Robison,MBARI,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,718.200012,718.200012,30.402252,-137.669863,NaN,320.899994,NaN,NaN,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,urn:tid:mbari.org:T0265-04,21.55
3,2004-05-04T00:27:02+00:00,Tiburon_0666,ROV,David Clague,MBARI,A3DE4416-5B67-4A10-A6BC-AE9A99552A13,Dosidicus gigas,vars,546.400024,546.400024,32.273680,-119.672170,0.380,70.800003,34.199001,5.580,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2004/124/03_25_01_14.png,urn:tid:mbari.org:T0666-04,16.39
4,2010-02-26T02:58:34+00:00,Ventana_3527,ROV,Linda Kuhnz,MBARI,BF43A883-1DE2-439A-8F55-F7F215FE2D4F,Dosidicus gigas,linda,441.980011,441.980011,36.797201,-122.188644,0.854,195.199997,34.175999,6.959,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3527/00_38_45_13.jpg,urn:tid:mbari.org:V3527-01HD,13.26
6,2007-11-03T03:22:18+00:00,Tiburon_1147,ROV,Bruce Robison,MBARI,0EFEFAFD-B9ED-4640-9873-75A5EDA7C253,Muusoctopus robustus,svonthun,3411.820068,3411.820068,36.323256,-122.903316,2.650,139.000000,34.484001,1.589,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/1147/06_40_40_00.jpg,urn:tid:mbari.org:T1147-07,102.35
7,2010-02-25T01:59:12+00:00,Ventana_3525,ROV,Chad Widmer,MBARI,2E2280EF-397A-4384-B94B-23B12A287175,Dosidicus gigas,lonny,454.769989,454.769989,36.714396,-121.999731,0.828,99.300003,34.181000,6.962,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3525/01_06_04_24.jpg,urn:tid:mbari.org:V3525-02HD,13.64


**Note** that the calculation for coordinateUncertaintyInMeters may be more complex once I've talked to Dave Caress. At the moment, I've just used a rule of thumb that Brian gave me that he may or may not have remembered correctly: that uncertainty goes like 3% of depth. Also, I've somewhat arbitrarily rounded to two decimal places. **It might be worth asking whether the depths, which are reported to 6 decimal places, are actually that accurate. Unlikely.**

In [68]:
## Get a list of unique species names

names = converted['scientificName'].unique()
names

array(['Dosidicus gigas', 'Muusoctopus robustus', 'Patellogastropoda',
       'Muusoctopus leioderma', 'Dosidicus', 'Arachnactis'], dtype=object)

In [69]:
%%time

## Call run_get_worms_from_scientific_name

name_id_dic, name_dic, id_dic = WoRMS.run_get_worms_from_scientific_name(names)

Wall time: 3.05 s


In [71]:
## Create columns from WoRMS data

# Create scientificNameID column with the same content as scientificName - strip to ensure no whitespace
converted['scientificNameID'] = converted['scientificName'].str.strip()

# Use dictionary to replace scientific names with name IDs
converted.replace({'scientificNameID':name_id_dic}, inplace=True)

# Repeat to create taxonID
converted['taxonID'] = converted['scientificName'].str.strip()
converted.replace({'taxonID':id_dic}, inplace=True)

# Create additional needed columns
converted['nameAccordingTo'] = 'WoRMS'
converted['occurrenceStatus'] = 'present'
converted['basisOfRecord'] = 'HumanObservation'

converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,minimumDepthInMeters,maximumDepthInMeters,...,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord
0,2001-03-20T05:24:12+00:00,Tiburon_0265,ROV,Bruce Robison,MBARI,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,718.200012,718.200012,...,NaN,NaN,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,urn:tid:mbari.org:T0265-04,21.55,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation
3,2004-05-04T00:27:02+00:00,Tiburon_0666,ROV,David Clague,MBARI,A3DE4416-5B67-4A10-A6BC-AE9A99552A13,Dosidicus gigas,vars,546.400024,546.400024,...,34.199001,5.580,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2004/124/03_25_01_14.png,urn:tid:mbari.org:T0666-04,16.39,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation
4,2010-02-26T02:58:34+00:00,Ventana_3527,ROV,Linda Kuhnz,MBARI,BF43A883-1DE2-439A-8F55-F7F215FE2D4F,Dosidicus gigas,linda,441.980011,441.980011,...,34.175999,6.959,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3527/00_38_45_13.jpg,urn:tid:mbari.org:V3527-01HD,13.26,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation
6,2007-11-03T03:22:18+00:00,Tiburon_1147,ROV,Bruce Robison,MBARI,0EFEFAFD-B9ED-4640-9873-75A5EDA7C253,Muusoctopus robustus,svonthun,3411.820068,3411.820068,...,34.484001,1.589,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/1147/06_40_40_00.jpg,urn:tid:mbari.org:T1147-07,102.35,urn:lsid:marinespecies.org:taxname:527163,527163,WoRMS,present,HumanObservation
7,2010-02-25T01:59:12+00:00,Ventana_3525,ROV,Chad Widmer,MBARI,2E2280EF-397A-4384-B94B-23B12A287175,Dosidicus gigas,lonny,454.769989,454.769989,...,34.181000,6.962,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3525/01_06_04_24.jpg,urn:tid:mbari.org:V3525-02HD,13.64,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation


In [72]:
## Finally, assemble associatedMedia

selected = converted[converted['occurrenceID'] == '5815263A-A69E-48B4-A5C7-A1E17599EEC3']
selected

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,minimumDepthInMeters,maximumDepthInMeters,...,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord
428,2009-05-07T08:49:47+00:00,Doc_Ricketts_0019,ROV,Bruce Robison,MBARI,5815263A-A69E-48B4-A5C7-A1E17599EEC3,Dosidicus gigas,svonthun,264.170013,264.170013,...,34.125,7.464,http://search.mbari.org/ARCHIVE/frameGrabs/Doc%20Ricketts/images/0019/05_36_01_06.jpg,urn:tid:mbari.org:D0019-06HD,7.93,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation
1852,2009-05-07T08:49:47+00:00,Doc_Ricketts_0019,ROV,Bruce Robison,MBARI,5815263A-A69E-48B4-A5C7-A1E17599EEC3,Dosidicus gigas,svonthun,264.170013,264.170013,...,34.125,7.464,http://search.mbari.org/ARCHIVE/frameGrabs/Doc%20Ricketts/images/0019/05_36_01_06.png,urn:tid:mbari.org:D0019-06HD,7.93,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation


In [73]:
if selected.shape[0] > 1:
    image_files = selected['image_url'].drop_duplicates()
    video_files = selected['video_uri'].drop_duplicates()
    
print(image_files)
video_files

428     http://search.mbari.org/ARCHIVE/frameGrabs/Doc%20Ricketts/images/0019/05_36_01_06.jpg
1852    http://search.mbari.org/ARCHIVE/frameGrabs/Doc%20Ricketts/images/0019/05_36_01_06.png
Name: image_url, dtype: object


428    urn:tid:mbari.org:D0019-06HD
Name: video_uri, dtype: object

In [74]:
media = pd.concat([image_files, video_files])
media

428     http://search.mbari.org/ARCHIVE/frameGrabs/Doc%20Ricketts/images/0019/05_36_01_06.jpg
1852    http://search.mbari.org/ARCHIVE/frameGrabs/Doc%20Ricketts/images/0019/05_36_01_06.png
428                                                              urn:tid:mbari.org:D0019-06HD
dtype: object

In [75]:
url_str = ''
for url in media:
    url_str = url_str + url + ' | '

url_str = url_str[0:-3]
url_str

'http://search.mbari.org/ARCHIVE/frameGrabs/Doc%20Ricketts/images/0019/05_36_01_06.jpg | http://search.mbari.org/ARCHIVE/frameGrabs/Doc%20Ricketts/images/0019/05_36_01_06.png | urn:tid:mbari.org:D0019-06HD'

In [77]:
## Ok, so let's do that for all occurrenceID's

associatedMedia = []

for occ_id in converted['occurrenceID'].unique():
    
    # Select data associated with that occurrenceID:
    selected = converted[converted['occurrenceID'] == occ_id]
    
    # Retrieve unique image and video files
    image_files = selected['image_url'].drop_duplicates()
    video_files = selected['video_uri'].drop_duplicates()
    
    # Remove any NaN values
    image_files = image_files.dropna()
    video_files = video_files.dropna()
    
    # Join image and video files
    media = pd.concat([image_files, video_files])
    
    # Create a string with all the urls
    url_str = ''
    for url in media: url_str = url_str + url + ' | '
    url_str = url_str[0:-3]
    
    # Add to associatedMedia
    associatedMedia.append(url_str)

    

In [78]:
## Add to df

# First, need to remove rows with duplicate occurrenceIDs
converted = converted.drop_duplicates(subset='occurrenceID', keep="first")

# Add associatedMedia
converted['associatedMedia'] = associatedMedia
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,minimumDepthInMeters,maximumDepthInMeters,...,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord,associatedMedia
0,2001-03-20T05:24:12+00:00,Tiburon_0265,ROV,Bruce Robison,MBARI,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,718.200012,718.200012,...,NaN,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg,urn:tid:mbari.org:T0265-04,21.55,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg | http://sear...
3,2004-05-04T00:27:02+00:00,Tiburon_0666,ROV,David Clague,MBARI,A3DE4416-5B67-4A10-A6BC-AE9A99552A13,Dosidicus gigas,vars,546.400024,546.400024,...,5.580,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2004/124/03_25_01_14.png,urn:tid:mbari.org:T0666-04,16.39,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2004/124/03_25_01_14.png | http://sear...
4,2010-02-26T02:58:34+00:00,Ventana_3527,ROV,Linda Kuhnz,MBARI,BF43A883-1DE2-439A-8F55-F7F215FE2D4F,Dosidicus gigas,linda,441.980011,441.980011,...,6.959,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3527/00_38_45_13.jpg,urn:tid:mbari.org:V3527-01HD,13.26,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3527/00_38_45_13.jpg | http://search.m...
6,2007-11-03T03:22:18+00:00,Tiburon_1147,ROV,Bruce Robison,MBARI,0EFEFAFD-B9ED-4640-9873-75A5EDA7C253,Muusoctopus robustus,svonthun,3411.820068,3411.820068,...,1.589,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/1147/06_40_40_00.jpg,urn:tid:mbari.org:T1147-07,102.35,urn:lsid:marinespecies.org:taxname:527163,527163,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/1147/06_40_40_00.jpg | http://search.m...
7,2010-02-25T01:59:12+00:00,Ventana_3525,ROV,Chad Widmer,MBARI,2E2280EF-397A-4384-B94B-23B12A287175,Dosidicus gigas,lonny,454.769989,454.769989,...,6.962,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3525/01_06_04_24.jpg,urn:tid:mbari.org:V3525-02HD,13.64,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3525/01_06_04_24.jpg | http://search.m...


In [79]:
converted.columns

Index(['eventDate', 'eventID', 'samplingProtocol', 'recordedBy',
       'institutionCode', 'occurrenceID', 'scientificName', 'identifiedBy',
       'minimumDepthInMeters', 'maximumDepthInMeters', 'decimalLatitude',
       'decimalLongitude', 'dissolvedOxygenInMLPerL', 'pressureInPsi',
       'salinity', 'temperatureInCelsius', 'image_url', 'video_uri',
       'coordinateUncertaintyInMeters', 'scientificNameID', 'taxonID',
       'nameAccordingTo', 'occurrenceStatus', 'basisOfRecord',
       'associatedMedia'],
      dtype='object')

In [80]:
## Drop extra columns

converted = converted.drop(['image_url', 'video_uri'], axis=1)
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,identifiedBy,minimumDepthInMeters,maximumDepthInMeters,...,pressureInPsi,salinity,temperatureInCelsius,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord,associatedMedia
0,2001-03-20T05:24:12+00:00,Tiburon_0265,ROV,Bruce Robison,MBARI,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,schlin,718.200012,718.200012,...,320.899994,NaN,NaN,21.55,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg | http://sear...
3,2004-05-04T00:27:02+00:00,Tiburon_0666,ROV,David Clague,MBARI,A3DE4416-5B67-4A10-A6BC-AE9A99552A13,Dosidicus gigas,vars,546.400024,546.400024,...,70.800003,34.199001,5.580,16.39,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2004/124/03_25_01_14.png | http://sear...
4,2010-02-26T02:58:34+00:00,Ventana_3527,ROV,Linda Kuhnz,MBARI,BF43A883-1DE2-439A-8F55-F7F215FE2D4F,Dosidicus gigas,linda,441.980011,441.980011,...,195.199997,34.175999,6.959,13.26,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3527/00_38_45_13.jpg | http://search.m...
6,2007-11-03T03:22:18+00:00,Tiburon_1147,ROV,Bruce Robison,MBARI,0EFEFAFD-B9ED-4640-9873-75A5EDA7C253,Muusoctopus robustus,svonthun,3411.820068,3411.820068,...,139.000000,34.484001,1.589,102.35,urn:lsid:marinespecies.org:taxname:527163,527163,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/1147/06_40_40_00.jpg | http://search.m...
7,2010-02-25T01:59:12+00:00,Ventana_3525,ROV,Chad Widmer,MBARI,2E2280EF-397A-4384-B94B-23B12A287175,Dosidicus gigas,lonny,454.769989,454.769989,...,99.300003,34.181000,6.962,13.64,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,present,HumanObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3525/01_06_04_24.jpg | http://search.m...


In [81]:
## Reorder columns

converted = converted[['eventID', 'eventDate', 'samplingProtocol', 'recordedBy', 'institutionCode', 'occurrenceID', 'scientificName', 'scientificNameID', 'taxonID', 
                       'nameAccordingTo', 'occurrenceStatus', 'basisOfRecord', 'identifiedBy', 'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters',
                       'minimumDepthInMeters', 'maximumDepthInMeters', 'dissolvedOxygenInMLPerL', 'pressureInPsi', 'salinity', 'temperatureInCelsius', 'associatedMedia']]
converted.head()

,eventID,eventDate,samplingProtocol,recordedBy,institutionCode,occurrenceID,scientificName,scientificNameID,taxonID,nameAccordingTo,...,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,minimumDepthInMeters,maximumDepthInMeters,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,associatedMedia
0,Tiburon_0265,2001-03-20T05:24:12+00:00,ROV,Bruce Robison,MBARI,1040A0BF-2D50-45D2-9C96-35CC80708939,Dosidicus gigas,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,...,30.402252,-137.669863,21.55,718.200012,718.200012,NaN,320.899994,NaN,NaN,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2001/078/04_00_48_12.jpg | http://sear...
3,Tiburon_0666,2004-05-04T00:27:02+00:00,ROV,David Clague,MBARI,A3DE4416-5B67-4A10-A6BC-AE9A99552A13,Dosidicus gigas,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,...,32.273680,-119.672170,16.39,546.400024,546.400024,0.380,70.800003,34.199001,5.580,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/stills/2004/124/03_25_01_14.png | http://sear...
4,Ventana_3527,2010-02-26T02:58:34+00:00,ROV,Linda Kuhnz,MBARI,BF43A883-1DE2-439A-8F55-F7F215FE2D4F,Dosidicus gigas,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,...,36.797201,-122.188644,13.26,441.980011,441.980011,0.854,195.199997,34.175999,6.959,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3527/00_38_45_13.jpg | http://search.m...
6,Tiburon_1147,2007-11-03T03:22:18+00:00,ROV,Bruce Robison,MBARI,0EFEFAFD-B9ED-4640-9873-75A5EDA7C253,Muusoctopus robustus,urn:lsid:marinespecies.org:taxname:527163,527163,WoRMS,...,36.323256,-122.903316,102.35,3411.820068,3411.820068,2.650,139.000000,34.484001,1.589,http://search.mbari.org/ARCHIVE/frameGrabs/Tiburon/images/1147/06_40_40_00.jpg | http://search.m...
7,Ventana_3525,2010-02-25T01:59:12+00:00,ROV,Chad Widmer,MBARI,2E2280EF-397A-4384-B94B-23B12A287175,Dosidicus gigas,urn:lsid:marinespecies.org:taxname:342291,342291,WoRMS,...,36.714396,-121.999731,13.64,454.769989,454.769989,0.828,99.300003,34.181000,6.962,http://search.mbari.org/ARCHIVE/frameGrabs/Ventana/images/3525/01_06_04_24.jpg | http://search.m...


### Save

In [82]:
## Save

converted.to_csv('VARS_six-species_converted.csv', index=False, na_rep='NaN')

### Questions for Brian
- Who do I need to talk to to confirm that coordinates are in WGS84? Do we have any sense of the accuracy of those coordinates? **Contact Dave Caress. Accuracy may go like 3% of the depth.**
- How persistent are the image url's/uri's? Double check that images are accessible to everyone. **They are NOT accessible to folks outside MBARI. The links are permanent, but if we choose to make the resources available to folks outside MBARI the links will change.**
- Following up: JayDeBeApi doesn't seem to be doing anything weird to the date times - so possibly fixed? They match what I'm getting using the VARS Query App and what's on the actual images. **Sounds like it's fixed**